In [1]:
!pip install langchain langchainhub langchain-openai langchain-community faiss-cpu


[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
# openai api key
os.environ['OPENAI_API_KEY'] = "XXXXXXXX"
# tavily api key
os.environ['TAVILY_API_KEY'] = "XXXXXXXX"

In [3]:
# web based documents loader
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()

In [4]:
# openai embedding
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [5]:
# build documents into vectors
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [6]:
# retriever
retriever = vector.as_retriever()

In [7]:
# retriever tool
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [8]:
# search
from langchain_community.tools.tavily_search import TavilySearchResults
search = TavilySearchResults()

In [9]:
# tool chain
tools = [retriever_tool, search]

In [10]:
# prompt, llm, agent
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

prompt = hub.pull("hwchase17/openai-functions-agent")
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
# invoke
print(agent_executor.invoke({"input": "how can langsmith help with testing?"}))
print(agent_executor.invoke({"input": "what is the weather in SF?"}))

from langchain_core.messages import HumanMessage, AIMessage
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]

print(agent_executor.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
}))



> Entering new AgentExecutor chain...
LangSmith can help with testing in several ways:

1. Test Case Generation: LangSmith can generate test cases automatically based on the specifications of the software being tested. It uses advanced techniques such as symbolic execution and constraint solving to generate test inputs that cover different execution paths and edge cases.

2. Test Oracles: LangSmith can help in defining test oracles, which are the expected outputs or behaviors of the software under test. It can analyze the specifications and requirements of the software to automatically generate assertions or checks that can be used as test oracles.

3. Test Coverage Analysis: LangSmith can analyze the coverage of test cases to determine which parts of the software have been exercised during testing. It can provide metrics and visualizations to help testers understand the effectiveness of their test suite and identify areas that need more testing.

4. Test Prioritization: LangSmith ca